<a href="https://colab.research.google.com/github/Praxis-QR/RDWH/blob/main/Window_Functions_MySQL_PostgreSQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![alt text](https://github.com/Praxis-QR/RDWH/raw/main/images/YantraJaalBanner.png)<br>


<hr>

[Prithwis Mukerjee](http://www.linkedin.com/in/prithwis)<br>

#Install and SetUp

## MySQL

In [1]:
!apt-get update > /dev/null
!apt-get -y install mysql-server > /dev/null
!/etc/init.d/mysql restart
!mysql --version

 * Stopping MySQL database server mysqld
   ...done.
 * Starting MySQL database server mysqld
su: warning: cannot change directory to /nonexistent: No such file or directory
   ...done.


In [3]:
!mysql -e 'DROP DATABASE IF EXISTS praxisdb;'
!mysql -e 'CREATE DATABASE praxisdb;'
!mysql praxisdb -e 'show tables;'

In [ ]:
#!mysql praxisdb -e 'desc product_groups'

##PostgreSQL

In [1]:
# Install postgresql server
!apt update > /dev/null
!apt install postgresql > /dev/null

!service postgresql start





 * Starting PostgreSQL 12 database server
   ...done.


In [2]:
!sudo -u postgres psql -V

psql (PostgreSQL) 12.13 (Ubuntu 12.13-0ubuntu0.20.04.1)


In [3]:
# Setup a password `postgres` for username `postgres`
!sudo -u postgres psql -U postgres -c "ALTER USER postgres PASSWORD 'pass';"
#
# Setup a database with name `sampledb` to be used
!sudo -u postgres psql -U postgres -c 'DROP DATABASE IF EXISTS praxisdb;'
!sudo -u postgres psql -U postgres -c 'CREATE DATABASE praxisdb;'

ALTER ROLE
NOTICE:  database "praxisdb" does not exist, skipping
DROP DATABASE
CREATE DATABASE


In [4]:
# Create file to store the password. Else will be prompted each time
!echo "localhost:5432:praxisdb:postgres:pass" > ~/.pgpass
!chmod 0600 ~/.pgpass
!cat ~/.pgpass

localhost:5432:praxisdb:postgres:pass


## Python Connection

In [5]:
import pandas as pd

!apt install libmysqlclient-dev > /dev/null
!pip install mysqlclient > /dev/null
import MySQLdb

#!pip install psycopg2
import psycopg2


In [6]:
conn = psycopg2.connect(database="praxisdb", user='postgres', password='pass', host='127.0.0.1', port= '5432')

In [7]:
def runSelect(s):
    #Set up connection
    conn = MySQLdb.connect(db='praxisdb')
    #conn = psycopg2.connect(database="praxisdb", user='postgres', password='pass', host='127.0.0.1', port= '5432')
    resultdf = pd.read_sql(s, con=conn)
    conn.close()
    return resultdf

def runCmd(s):
    #Create Connection
    conn = MySQLdb.connect(db='praxisdb')
    #conn = psycopg2.connect(database="praxisdb", user='postgres', password='pass', host='127.0.0.1', port= '5432')
    cursor = conn.cursor()
    try:
        cursor.execute(s)
        print('Command executed')
        conn.commit()
    except:
        print('Command failed')
    #Closing the connection
    conn.close()

In [ ]:
#!ps -aef

# Create, Load Tables

In [8]:
ddl = 'drop table if exists product_groups;'
runCmd(ddl)
ddl = 'drop table if exists products;'
runCmd(ddl)

Command executed
Command executed


In [9]:
ddl = "\
CREATE TABLE product_groups (\
	group_id char(3) PRIMARY KEY,\
	group_name VARCHAR (255) NOT NULL\
);"
runCmd(ddl)

ddl = "\
CREATE TABLE products (\
	product_id char(3) PRIMARY KEY,\
	product_name VARCHAR (255) NOT NULL,\
	price DECIMAL (11, 2),\
	group_id char(3) NOT NULL,\
	FOREIGN KEY (group_id) REFERENCES product_groups (group_id)\
);"
runCmd(ddl)


Command executed
Command executed


In [10]:
ddl = "\
INSERT INTO product_groups (group_id,group_name) \
VALUES \
	('SP1','Smartphone'), \
	('LT2','Laptop'), \
	('TB3','Tablet'); \
"
runCmd(ddl)
ddl = "\
INSERT INTO products (product_id,product_name, group_id,price) \
VALUES \
	('P11','Microsoft Lumia', 'SP1', 200),\
	('P12','HTC One', 'SP1', 400),\
	('P13','Nexus', 'SP1', 500),\
	('P14','iPhone', 'SP1', 900),\
	('P21','HP Elite', 'LT2', 1200),\
	('P22','Lenovo Thinkpad', 'LT2', 700),\
	('P23','Sony VAIO', 'LT2', 700),\
	('P24','Dell Vostro', 'LT2', 800),\
	('P31','iPad', 'TB3', 700),\
	('P32','Kindle Fire', 'TB3', 150),\
	('P33','Samsung Galaxy Tab', 'TB3', 200);"

runCmd(ddl)

Command executed
Command executed


In [55]:
runSelect("select * from products,product_groups \
           where products.group_id = product_groups.group_id")

,product_id,product_name,price,group_id,group_id,group_name
0,P21,HP Elite,1200.0,LT2,LT2,Laptop
1,P22,Lenovo Thinkpad,700.0,LT2,LT2,Laptop
2,P23,Sony VAIO,700.0,LT2,LT2,Laptop
3,P24,Dell Vostro,800.0,LT2,LT2,Laptop
4,P11,Microsoft Lumia,200.0,SP1,SP1,Smartphone
5,P12,HTC One,400.0,SP1,SP1,Smartphone
6,P13,Nexus,500.0,SP1,SP1,Smartphone
7,P14,iPhone,900.0,SP1,SP1,Smartphone
8,P31,iPad,700.0,TB3,TB3,Tablet
9,P32,Kindle Fire,150.0,TB3,TB3,Tablet


#Windows Functions 
explanation for the SQL used here is available at this URL 
https://www.postgresqltutorial.com/postgresql-window-function/

In [12]:
runSelect("select * from products")

,product_id,product_name,price,group_id
0,P11,Microsoft Lumia,200.0,SP1
1,P12,HTC One,400.0,SP1
2,P13,Nexus,500.0,SP1
3,P14,iPhone,900.0,SP1
4,P21,HP Elite,1200.0,LT2
5,P22,Lenovo Thinkpad,700.0,LT2
6,P23,Sony VAIO,700.0,LT2
7,P24,Dell Vostro,800.0,LT2
8,P31,iPad,700.0,TB3
9,P32,Kindle Fire,150.0,TB3


In [13]:
sql = "\
SELECT 	round(AVG (price),2) avgPrice \
FROM 	products; \
"
runSelect(sql)

,avgprice
0,586.36


In [14]:
sql = "\
SELECT 	group_id, round(AVG (price),2) avgPrice \
FROM 	products \
group by group_id \
"
runSelect(sql)

,group_id,avgprice
0,LT2,850.0
1,SP1,500.0
2,TB3,350.0


In [24]:
sql = "\
SELECT 	 group_id, AVG (price)   \
FROM 	products \
group by group_id \
"
runSelect(sql)

,group_id,AVG (price)
0,LT2,850.0
1,SP1,500.0
2,TB3,350.0


In [58]:
sql = "\
SELECT 	  product_name, price, group_id, avg(price) over (partition by group_id) AvgPrice \
FROM products \
"
runSelect(sql)

,product_name,price,group_id,AvgPrice
0,HP Elite,1200.0,LT2,850.0
1,Lenovo Thinkpad,700.0,LT2,850.0
2,Sony VAIO,700.0,LT2,850.0
3,Dell Vostro,800.0,LT2,850.0
4,Microsoft Lumia,200.0,SP1,500.0
5,HTC One,400.0,SP1,500.0
6,Nexus,500.0,SP1,500.0
7,iPhone,900.0,SP1,500.0
8,iPad,700.0,TB3,350.0
9,Kindle Fire,150.0,TB3,350.0


In [16]:
sql = "\
SELECT 	group_name, 	round(AVG (price),2) avgPrice \
FROM 	products INNER JOIN product_groups USING (group_id) \
GROUP BY 	group_name;\
"
runSelect(sql)

,group_name,avgprice
0,Smartphone,500.0
1,Tablet,350.0
2,Laptop,850.0


In [17]:
sql = "\
SELECT\
	product_name, price, group_name,\
	AVG (price)  OVER (PARTITION BY group_name) AVGprice \
    FROM products	INNER JOIN 	product_groups USING (group_id);\
"
runSelect(sql)

,product_name,price,group_name,avgprice
0,HP Elite,1200.0,Laptop,850.0
1,Lenovo Thinkpad,700.0,Laptop,850.0
2,Sony VAIO,700.0,Laptop,850.0
3,Dell Vostro,800.0,Laptop,850.0
4,Microsoft Lumia,200.0,Smartphone,500.0
5,HTC One,400.0,Smartphone,500.0
6,Nexus,500.0,Smartphone,500.0
7,iPhone,900.0,Smartphone,500.0
8,iPad,700.0,Tablet,350.0
9,Kindle Fire,150.0,Tablet,350.0


In [ ]:
sql = "\
SELECT 	product_name, group_name, price,\
	ROW_NUMBER () OVER (PARTITION BY group_name	ORDER BY price) RowNumber \
    FROM products INNER JOIN product_groups USING (group_id);\
"
runSelect(sql)

,product_name,group_name,price,RowNumber
0,Sony VAIO,Laptop,700.0,1
1,Lenovo Thinkpad,Laptop,700.0,2
2,Dell Vostro,Laptop,800.0,3
3,HP Elite,Laptop,1200.0,4
4,Microsoft Lumia,Smartphone,200.0,1
5,HTC One,Smartphone,400.0,2
6,Nexus,Smartphone,500.0,3
7,iPhone,Smartphone,900.0,4
8,Kindle Fire,Tablet,150.0,1
9,Samsung Galaxy Tab,Tablet,200.0,2


In [ ]:
sql = "\
SELECT 	product_name, 	group_name,   price, \
	RANK () OVER ( PARTITION BY group_name 	ORDER BY price 	) prodRank \
FROM products INNER JOIN product_groups USING (group_id); \
"
runSelect(sql)

,product_name,group_name,price,prodRank
0,Sony VAIO,Laptop,700.0,1
1,Lenovo Thinkpad,Laptop,700.0,1
2,Dell Vostro,Laptop,800.0,3
3,HP Elite,Laptop,1200.0,4
4,Microsoft Lumia,Smartphone,200.0,1
5,HTC One,Smartphone,400.0,2
6,Nexus,Smartphone,500.0,3
7,iPhone,Smartphone,900.0,4
8,Kindle Fire,Tablet,150.0,1
9,Samsung Galaxy Tab,Tablet,200.0,2


In [ ]:
sql = "\
SELECT 	product_name, 	group_name,   price, \
	DENSE_RANK () OVER ( PARTITION BY group_name 	ORDER BY price ) denseRank \
FROM products INNER JOIN product_groups USING (group_id); \
"
runSelect(sql)

,product_name,group_name,price,denseRank
0,Sony VAIO,Laptop,700.0,1
1,Lenovo Thinkpad,Laptop,700.0,1
2,Dell Vostro,Laptop,800.0,2
3,HP Elite,Laptop,1200.0,3
4,Microsoft Lumia,Smartphone,200.0,1
5,HTC One,Smartphone,400.0,2
6,Nexus,Smartphone,500.0,3
7,iPhone,Smartphone,900.0,4
8,Kindle Fire,Tablet,150.0,1
9,Samsung Galaxy Tab,Tablet,200.0,2


In [ ]:
sql = "\
SELECT 	product_name, 	group_name, 	price, \
	FIRST_VALUE (price) OVER (PARTITION BY group_name ORDER BY price) AS lowest_price_per_group , \
    LAST_VALUE (price) OVER (PARTITION BY group_name ORDER BY price \
	RANGE BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING \
	) AS highest_price_per_group \
FROM products INNER JOIN product_groups USING (group_id); \
"

runSelect(sql)


,product_name,group_name,price,lowest_price_per_group,highest_price_per_group
0,Sony VAIO,Laptop,700.0,700.0,1200.0
1,Lenovo Thinkpad,Laptop,700.0,700.0,1200.0
2,Dell Vostro,Laptop,800.0,700.0,1200.0
3,HP Elite,Laptop,1200.0,700.0,1200.0
4,Microsoft Lumia,Smartphone,200.0,200.0,900.0
5,HTC One,Smartphone,400.0,200.0,900.0
6,Nexus,Smartphone,500.0,200.0,900.0
7,iPhone,Smartphone,900.0,200.0,900.0
8,Kindle Fire,Tablet,150.0,150.0,700.0
9,Samsung Galaxy Tab,Tablet,200.0,150.0,700.0


In [ ]:
sql = "\
SELECT 	product_name, 	group_name, 	price, \
	LAG (price, 1) OVER (PARTITION BY group_name ORDER BY price ) AS prev_price, \
	price - LAG (price, 1) OVER ( PARTITION BY group_name ORDER BY price) AS cur_prev_diff \
FROM 	products INNER JOIN product_groups USING (group_id); \
"
runSelect(sql)

,product_name,group_name,price,prev_price,cur_prev_diff
0,Sony VAIO,Laptop,700.0,NaN,NaN
1,Lenovo Thinkpad,Laptop,700.0,700.0,0.0
2,Dell Vostro,Laptop,800.0,700.0,100.0
3,HP Elite,Laptop,1200.0,800.0,400.0
4,Microsoft Lumia,Smartphone,200.0,NaN,NaN
5,HTC One,Smartphone,400.0,200.0,200.0
6,Nexus,Smartphone,500.0,400.0,100.0
7,iPhone,Smartphone,900.0,500.0,400.0
8,Kindle Fire,Tablet,150.0,NaN,NaN
9,Samsung Galaxy Tab,Tablet,200.0,150.0,50.0


## All the functions!

In [79]:
sql = "\
SELECT 	  product_name, price, group_id, \
avg(price) over (partition by group_id) AvgPrice, \
ROW_NUMBER ()  over (partition by group_id) RowNum, \
RANK () OVER ( PARTITION BY group_id ORDER BY price) prodRank, \
DENSE_RANK () OVER ( PARTITION BY group_id ORDER BY price) denseRank, \
PERCENT_RANK () OVER ( PARTITION BY group_id ORDER BY price) pctRank, \
FIRST_VALUE (price) OVER (PARTITION BY group_id ORDER BY price) AS groupLowest , \
LAST_VALUE (price) OVER (PARTITION BY group_id ORDER BY price) as groupHighest, \
NTH_VALUE (price,3) OVER (PARTITION BY group_id ORDER BY price) as Nth, \
LAG (price,1) OVER (PARTITION BY group_id ORDER BY price) as prevPrice, \
LEAD (price,1) OVER (PARTITION BY group_id ORDER BY price) as nextPrice \
FROM products \
"
runSelect(sql)

,product_name,price,group_id,AvgPrice,RowNum,prodRank,denseRank,pctRank,groupLowest,groupHighest,Nth,prevPrice,nextPrice
0,Lenovo Thinkpad,700.0,LT2,850.0,2,1,1,0.000000,700.0,700.0,NaN,NaN,700.0
1,Sony VAIO,700.0,LT2,850.0,3,1,1,0.000000,700.0,700.0,NaN,700.0,800.0
2,Dell Vostro,800.0,LT2,850.0,4,3,2,0.666667,700.0,800.0,800.0,700.0,1200.0
3,HP Elite,1200.0,LT2,850.0,1,4,3,1.000000,700.0,1200.0,800.0,800.0,NaN
4,Microsoft Lumia,200.0,SP1,500.0,1,1,1,0.000000,200.0,200.0,NaN,NaN,400.0
5,HTC One,400.0,SP1,500.0,2,2,2,0.333333,200.0,400.0,NaN,200.0,500.0
6,Nexus,500.0,SP1,500.0,3,3,3,0.666667,200.0,500.0,500.0,400.0,900.0
7,iPhone,900.0,SP1,500.0,4,4,4,1.000000,200.0,900.0,500.0,500.0,NaN
8,Kindle Fire,150.0,TB3,350.0,2,1,1,0.000000,150.0,150.0,NaN,NaN,200.0
9,Samsung Galaxy Tab,200.0,TB3,350.0,3,2,2,0.500000,150.0,200.0,NaN,150.0,700.0


In [ ]:
from datetime import datetime
import pytz
print('Last tested  ',datetime.now(pytz.timezone('Asia/Kolkata')))

Last tested   2023-01-27 11:51:51.779709+05:30


#Chronobooks <br>
Two science fiction novels by Prithwis Mukerjee. A dystopian Earth. A technocratic society managed by artificial intelligence. Escape and epiphany on Mars. Can man and machine, carbon and silicon explore and escape into other dimensions of existence? An Indic perspective rooted in Advaita Vedanta and the Divine Feminine.  [More information](http://bit.ly/chronobooks) <br>
![alt text](https://github.com/Praxis-QR/RDWH/raw/main/images/CTCYFooter-1.png)